<img src="../images/felipepelaquim-9c7CK4BemmA-unsplash.jpg"  style="float: center; margin: 0px; height: 350px; width: 1000px"> 

# Afrobeats influence in Hip Hop

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import random
import time

In [2]:
#Authentication
client_credentials_manager = SpotifyClientCredentials(client_id = '2101cd224f5948e19c4c782d76744ed3',
                                                      client_secret = '0f81ac4a6af148138227fe1ee70ee64d')

Create spotipy object using SpotifyClientCredentials

In [3]:
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [4]:
#tracks = sp.playlist_tracks("5ZCzd0nCLqiIX1jwQWfazW")

results = sp.playlist_tracks('5ZCzd0nCLqiIX1jwQWfazW')
tracks = results['items']
while results['next']:
    results = sp.next(results)
    tracks.extend(results['items'])

In [7]:
type(tracks)

list

In [31]:
info = []
for track in tracks["items"]:
        info.append((track["track"]["uri"]).split(':')[2])

TypeError: list indices must be integers or slices, not str

In [6]:
tracks['items'][0]['track'].keys()

TypeError: list indices must be integers or slices, not str

## Get song info

Create a function that takes in a playlist of songs and extracts the songs information and features such as artist name, genre, album name, track_uri, danceability, energy, loudness, instrumentalness etc

In [8]:
def get_track_info(playlist):
    
    #split the playlist_uri is at the end of the playlists url. I'll use the .split method to extract it
    uri = playlist.split("/")[-1].split("?")[0]
    
    #from the spotipy library, use the playlist_tracks() method to extract each track from the playlist uri
    #It comes in a nested dictionary format
    
    results = sp.playlist_tracks(uri)
    tracks = results['items']
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    
    #create an empty dictionary with the info we want to extract as columns
    info = {
    'track_uri':[],
    'track_name':[],
    'artist_name':[],
    'artist_info':[],
    'artist_uri':[],
    'artist_popularity':[],
    'artist_genre':[],
    'album':[],
    'track_pop':[],
    }
    
    features = {'danceability': [],
     'energy': [],
     'key': [],
     'loudness': [],
     'mode': [],
     'speechiness': [],
     'acousticness': [],
     'instrumentalness': [],
     'liveness': [],
     'valence': [],
     'tempo': [],
     'type': [],
     'id': [],
     'uri': [],
     'track_href': [],
     'analysis_url': [],
     'duration_ms': [],
     'time_signature': []
               }
    
    #using a for loop, get the the info for each song and put it into the empty dictionary
    for track in tracks:
        #URI
        info['track_uri'].append((track["track"]["uri"]).split(':')[2])

        #Track name
        info['track_name'].append(track["track"]["name"])

        #Main Artist
        info['artist_uri'].append((track["track"]["artists"][0]["uri"]).split(':')[2])
        info['artist_info'].append(sp.artist(track["track"]["artists"][0]["uri"]))

        #Name, popularity, genre
        info['artist_name'].append(track["track"]["artists"][0]["name"])
        info['artist_popularity'].append(sp.artist(track["track"]["artists"][0]["uri"])["popularity"])
        info['artist_genre'].append(sp.artist(track["track"]["artists"][0]["uri"])["genres"])

        #Album
        info['album'].append(track["track"]["album"]["name"])

        #Popularity of the track
        info['track_pop'].append(track["track"]["popularity"])
        
        
        
        #loop through the tracks to their features and assign it to the empty dictionary
        
        track_uri = track["track"]["uri"].split(':')[2] 
        
        for key,value in (sp.audio_features(track_uri)[0]).items():
            features[key].append(value)
            
        #time
        time.sleep(random.uniform(1,5))
        
    #Transform the features dictionary into a dataframe
    features_df = pd.DataFrame(features)
    
    #drop features we do not need
    features_df.drop(columns=['type','id','uri','track_href','analysis_url','duration_ms','time_signature'], inplace=True)   
    
    #Transform the info dictionary into a dataframe
    info_df = pd.DataFrame(info)
    
     
    
    return info_df.join(features_df)
        
        
        

#### Using the function created get features for Afrobeats Top playlist 

In [9]:
afrobeats_playlist_url= "https://open.spotify.com/playlist/5ZCzd0nCLqiIX1jwQWfazW"

In [ ]:
afrobeats_all_info = get_track_info(afrobeats_playlist_url)

In [70]:
afrobeats_all_info.head(2)

,track_uri,track_name,artist_name,artist_info,artist_uri,artist_popularity,artist_genre,album,track_pop,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0mDRuZmReEm6DquPLJlcEm,Oleku (feat. Brymo),Ice Prince,{'external_urls': {'spotify': 'https://open.sp...,1sSt1DqqqFLkPwfrqafVyn,54,"[afro dancehall, afropop, azontobeats, nigeria...",Oleku (feat. Brymo),51,0.544,0.730,1,-4.299,0,0.1440,0.344,0.000001,0.140,0.751,162.112
1,1tvi8tv0eykhNcV1WtaIqO,Move Back,5five,{'external_urls': {'spotify': 'https://open.sp...,37zb1JQnDV9dRLatrASEj1,29,"[afro dancehall, azonto, hiplife]",Move Back,45,0.793,0.734,11,-9.252,0,0.0981,0.323,0.000002,0.777,0.793,125.032


In [71]:
afrobeats_all_info.shape

(100, 20)

#### Using the function created get features for the playlist created by my classmates

The playlist is called dsi_0124

In [56]:
dsi_0124_playlist_url = "https://open.spotify.com/playlist/2ZRaoKtdIYxsQbLh7xCHvv?si=S20bfvXfTSCNkiI-FRynqw&nd=1"

In [57]:
dsi_0124_all_info = get_track_info(dsi_0124_playlist_url)

In [72]:
dsi_0124_all_info.head(2)

,track_uri,track_name,artist_name,artist_info,artist_uri,artist_popularity,artist_genre,album,track_pop,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1FUUFMaoXeRUb2SwVyx1D1,Hokey Pokey,Toddler Songs Kids,{'external_urls': {'spotify': 'https://open.sp...,76lfjOzsF7ZKDKj3YqzwfM,53,[children's music],Toddler Songs,48,0.793,0.497,7,-5.679,1,0.0275,0.365,0.0,0.1490,0.665,110.021
1,6btdYzQ8eZFBrOlUKVHuz0,"Be Our Guest - From ""Beauty and the Beast""/Sou...",Angela Lansbury,{'external_urls': {'spotify': 'https://open.sp...,0LtVJXnPR8msCJiE2DjHxy,60,"[hollywood, movie tunes, show tunes]",Beauty and the Beast,63,0.517,0.421,11,-12.822,0,0.1720,0.813,0.0,0.0788,0.578,121.459


In [104]:
dsi_0124_all_info.shape

(90, 20)

#### Add the labels to the datafames

In [105]:
afrobeats_all_info.loc[:,'genre'] = 'afrobeats'

In [106]:
afrobeats_all_info.head()

,track_uri,track_name,artist_name,artist_info,artist_uri,artist_popularity,artist_genre,album,track_pop,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,0mDRuZmReEm6DquPLJlcEm,Oleku (feat. Brymo),Ice Prince,{'external_urls': {'spotify': 'https://open.sp...,1sSt1DqqqFLkPwfrqafVyn,54,"[afro dancehall, afropop, azontobeats, nigeria...",Oleku (feat. Brymo),51,0.544,...,1,-4.299,0,0.1440,0.344,0.000001,0.140,0.751,162.112,afrobeats
1,1tvi8tv0eykhNcV1WtaIqO,Move Back,5five,{'external_urls': {'spotify': 'https://open.sp...,37zb1JQnDV9dRLatrASEj1,29,"[afro dancehall, azonto, hiplife]",Move Back,45,0.793,...,11,-9.252,0,0.0981,0.323,0.000002,0.777,0.793,125.032,afrobeats
2,5YgyvTZmdkIfFOnj8ri8mZ,Pakurumo,WizKid,{'external_urls': {'spotify': 'https://open.sp...,3tVQdUvClmAT7URs9V3rsp,80,"[afro dancehall, afropop, azonto, nigerian hip...",Superstar,49,0.879,...,2,-5.412,1,0.0421,0.525,0.005760,0.268,0.967,119.990,afrobeats
3,2cWdhA4ZDBvYuHRKKIPT3k,The Matter (feat. Wizkid),Maleek Berry,{'external_urls': {'spotify': 'https://open.sp...,520qA5VGL9iI0SbmEnTVNg,60,"[afro dancehall, afropop, azontobeats, nigeria...",The Matter (feat. Wizkid),48,0.697,...,2,-6.782,1,0.2710,0.224,0.000000,0.336,0.808,103.070,afrobeats
4,4voOsb8qpJ569JaMiGAyrl,Slow Down (feat. Wiz Kid),R2Bees,{'external_urls': {'spotify': 'https://open.sp...,0LFsP7WPfu5inz9a1amcE4,52,"[afro dancehall, afropop, azonto, azontobeats,...",Refuse to Be Broke: Da Revolution 2,53,0.520,...,6,-3.006,0,0.3890,0.116,0.000000,0.249,0.704,135.550,afrobeats


In [107]:
dsi_0124_all_info.loc[:,'genre'] = 'random'

In [108]:
dsi_0124_all_info.head()

,track_uri,track_name,artist_name,artist_info,artist_uri,artist_popularity,artist_genre,album,track_pop,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,1FUUFMaoXeRUb2SwVyx1D1,Hokey Pokey,Toddler Songs Kids,{'external_urls': {'spotify': 'https://open.sp...,76lfjOzsF7ZKDKj3YqzwfM,53,[children's music],Toddler Songs,48,0.793,...,7,-5.679,1,0.0275,0.3650,0.000000,0.1490,0.6650,110.021,random
1,6btdYzQ8eZFBrOlUKVHuz0,"Be Our Guest - From ""Beauty and the Beast""/Sou...",Angela Lansbury,{'external_urls': {'spotify': 'https://open.sp...,0LtVJXnPR8msCJiE2DjHxy,60,"[hollywood, movie tunes, show tunes]",Beauty and the Beast,63,0.517,...,11,-12.822,0,0.1720,0.8130,0.000000,0.0788,0.5780,121.459,random
2,3G9Jpr6F14VM7nvXoWXtTI,"Mahler: Symphony No. 2 in C Minor ""Resurrectio...",Gustav Mahler,{'external_urls': {'spotify': 'https://open.sp...,2ANtgfhQkKpsW6EYSDqldz,57,"[classical, post-romantic era]","Mahler: Symphony No.2, ""Resurrection""",29,0.169,...,0,-22.740,0,0.0374,0.9390,0.903000,0.1030,0.0662,98.850,random
3,4NE3oWyUQI2b2NjyMyC8Bi,Turtles All the Way Down,Sturgill Simpson,{'external_urls': {'spotify': 'https://open.sp...,3vDpQbGnzRbRVirXlfQagB,66,"[alternative americana, kentucky roots, nashvi...",Metamodern Sounds in Country Music,63,0.553,...,4,-9.217,1,0.0396,0.3060,0.000578,0.1040,0.5850,91.700,random
4,2QSrwZ7iVknZgkfi4aD6cn,Saltwater,Geowulf,{'external_urls': {'spotify': 'https://open.sp...,01TQ6CLvPSOYGUqRQ3nWgZ,47,[modern dream pop],Great Big Blue,62,0.552,...,2,-5.811,1,0.0305,0.0006,0.363000,0.1150,0.7090,107.031,random


#### Concat both dataframes

In [112]:
df = pd.concat([afrobeats_all_info, dsi_0124_all_info])

In [113]:
df.reset_index(inplace=True, drop=True)

In [114]:
df

,track_uri,track_name,artist_name,artist_info,artist_uri,artist_popularity,artist_genre,album,track_pop,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,0mDRuZmReEm6DquPLJlcEm,Oleku (feat. Brymo),Ice Prince,{'external_urls': {'spotify': 'https://open.sp...,1sSt1DqqqFLkPwfrqafVyn,54,"[afro dancehall, afropop, azontobeats, nigeria...",Oleku (feat. Brymo),51,0.544,...,1,-4.299,0,0.1440,0.34400,0.000001,0.1400,0.7510,162.112,afrobeats
1,1tvi8tv0eykhNcV1WtaIqO,Move Back,5five,{'external_urls': {'spotify': 'https://open.sp...,37zb1JQnDV9dRLatrASEj1,29,"[afro dancehall, azonto, hiplife]",Move Back,45,0.793,...,11,-9.252,0,0.0981,0.32300,0.000002,0.7770,0.7930,125.032,afrobeats
2,5YgyvTZmdkIfFOnj8ri8mZ,Pakurumo,WizKid,{'external_urls': {'spotify': 'https://open.sp...,3tVQdUvClmAT7URs9V3rsp,80,"[afro dancehall, afropop, azonto, nigerian hip...",Superstar,49,0.879,...,2,-5.412,1,0.0421,0.52500,0.005760,0.2680,0.9670,119.990,afrobeats
3,2cWdhA4ZDBvYuHRKKIPT3k,The Matter (feat. Wizkid),Maleek Berry,{'external_urls': {'spotify': 'https://open.sp...,520qA5VGL9iI0SbmEnTVNg,60,"[afro dancehall, afropop, azontobeats, nigeria...",The Matter (feat. Wizkid),48,0.697,...,2,-6.782,1,0.2710,0.22400,0.000000,0.3360,0.8080,103.070,afrobeats
4,4voOsb8qpJ569JaMiGAyrl,Slow Down (feat. Wiz Kid),R2Bees,{'external_urls': {'spotify': 'https://open.sp...,0LFsP7WPfu5inz9a1amcE4,52,"[afro dancehall, afropop, azonto, azontobeats,...",Refuse to Be Broke: Da Revolution 2,53,0.520,...,6,-3.006,0,0.3890,0.11600,0.000000,0.2490,0.7040,135.550,afrobeats
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,5qOhZG6k5nLLJbj6wRtV4w,Let's Do It Together - Casati SuperFresh Remix,Melleefresh,{'external_urls': {'spotify': 'https://open.sp...,7CxdGFcZV0dgrz4D35aG6o,35,"[canadian electronic, electro house]","Let's Do It Together: The SuperFresh Remixes, ...",7,0.701,...,10,-7.164,0,0.0624,0.00932,0.038900,0.1330,0.7840,124.053,random
186,0FlNY5774AK5s45hUmU39m,Sé,Alisone,{'external_urls': {'spotify': 'https://open.sp...,4jREeAP6ogIlfy7c2ME4Y5,24,[],Líricamente Amor,23,0.514,...,2,-12.077,1,0.0370,0.59900,0.000042,0.1340,0.0945,177.601,random
187,4u3oEIbf2vPHIjSHMPd8GZ,Pelo de mar,Alisone,{'external_urls': {'spotify': 'https://open.sp...,4jREeAP6ogIlfy7c2ME4Y5,24,[],Líricamente Amor,23,0.581,...,0,-12.281,1,0.0324,0.23100,0.000000,0.2160,0.6810,167.771,random
188,4cbnYdrG5qsAvG23iXWut0,A mi lado,Alisone,{'external_urls': {'spotify': 'https://open.sp...,4jREeAP6ogIlfy7c2ME4Y5,24,[],Líricamente Amor,23,0.735,...,2,-14.010,1,0.0439,0.01230,0.000099,0.0986,0.1500,83.919,random


#### Export dataframe as a csv

In [116]:
df.to_csv('../data/raw.csv', index = False )